In [1]:
import Packetizer
from Packetizer import Bonus, Tossup, Packet #necessary classes
import numpy as np
import pandas as pd
import re

In [2]:
def load_csv(fname):
    '''
    Loads in a QEMs CSV
    Returns two lists: one for tossups and one for bonuses
    '''
    count = 0
    tu_lines = []
    b_lines = []
    tu_bool = True #used to trakc if in tossup part of CSV
    done_bool = False #boolean to mark if done reading CSV
    
    if fname[-4:] == '.csv':
        temp_name = fname
    else:
        temp_name = fname + '.csv'
    
    try:
        wp = open(temp_name) #file exists
    except FileNotFoundError:
        raise FileNotFoundError('CSV not found. Is {} a file in the current directory?'.format(temp_name))
        
    for line in wp:
        if count > 0:
            line = line.replace('Packet','')
        #print(line.split(',')[0])
        if line == '\n': #new line char
            if tu_bool == False:
                done_bool = True
                break
            else:
                tu_bool = False
        if tu_bool == True and done_bool == False: #in tossups
            tu_lines.append(line)
        elif tu_bool == False and done_bool == False: #in bonuses and not done
            b_lines.append(line)
        count +=1
    wp.close()
    b_lines.pop(0) #remove first element of bonus list
    return tu_lines, b_lines

def make_tu_b_csvs(tu_list, b_list, title, sort = True):
    '''
    Makes two .csvs: one for tossups and one for bonuses
    '''
    #write to .csv for Tossup and bonus
    wp_t = open(title + '_t.csv', 'w')
    wp_b = open(title + '_b.csv', 'w')
    wp_t.write('\n'.join(tu_list))
    wp_b.write('\n'.join(b_list))
    wp_t.close()
    wp_b.close()
    #Tossups
    t_df = pd.read_csv(title + '_t.csv') #load in csv
    t_df = t_df.drop(['Comments', 'Question Number'], axis = 1 ) #useless columns
    t_df = t_df.fillna('0') #some values are np.nan so need to have a good value for them
    if sort == True:
        t_df = t_df.sort_values(by = ['Packet'])
    #Bonuses
    b_df = pd.read_csv(title + '_b.csv')
    b_df = b_df.drop(['Comments', 'Question Number'], axis = 1 ) #useless columns
    b_df = b_df.fillna('0') #some values are np.nan so need to have a good value for them
    b_df = b_df.rename(columns = {'Unnamed: 10': 'Packet'}) #used in wrangling data
    b_df = b_df.astype({'Packet':'int32'}, errors = 'ignore') #from str to int32
    if sort == True:
        b_df = b_df.sort_values(by = ['Packet'])
    
    return t_df, b_df
    

In [3]:
def add_tossups(packet_list, t_df, template_bool = False):
    '''
    Adds tossups to packets in packet_list
    If tmeplate_bool == True, requires distribution loaded in for categorization
    '''
    for index, row in t_df.iterrows():
        text = row['Tossup Question']
        answer = row['Answer']
        author = row['Author']
        packet = row['Packet']
        t_id = row['Id']
        temp_tu = Tossup(text, answer, author, t_id, packet)
        category = row['Category']
        if template_bool == True:
            pass #for now
        packet_list[packet - 1].add(temp_tu.clean_html()) #goes from 1 to 0 indexing
    return packet_list

def add_bonues(packet_list, b_df, template_bool = False):
    '''
    Adds bonues to packets in packet_list
    If tmeplate_bool == True, requires distribution loaded in for categorization
    '''
    for index, row in b_df.iterrows():
        lead_in = row['Bonus Leadin']
        q_list = [row['Bonus Part 1'], row['Bonus Part 2'], row['Bonus Part 3']]
        answer_list = [row['Bonus Answer 1'] , row['Bonus Answer 2'], row['Bonus Answer 3']]
        author = row['Author']
        b_id = row['Id']
        packet = row['Packet']
        temp_b = Bonus(lead_in, q_list, answer_list, author, b_id, packet)
        category = row['Category']
        if template_bool == True:
            pass #for now
        packet_list[packet - 1].add(temp_b.clean_html())
    return packet_list

#need get category helper function

In [4]:
def main():
    '''main program to generate HTML Packet files'''
    #TITLE = input('Title of this set or project?')
    tu_lines, b_lines = load_csv('./Demo/demo_shw.csv')
    t_df, b_df = make_tu_b_csvs(tu_lines, b_lines, 'demo_shw') #TITLE will go in here
    packet_list = []
    #num_packets = int(input)('Number of packets:?') #must be int
    num_packets = 16
    for i in range(num_packets):
        temp_packet = Packet([], [], i+1)
        packet_list.append(temp_packet)
    packet_list = add_tossups(packet_list, t_df) #add tossups
    packet_list = add_bonues(packet_list, b_df)
    for pack in packet_list:
        pack.randomize()
        pack.to_html() #dumps in current working directory as PacketNUMBER.html
    
    
    print('program done')
    return 0

In [5]:
if __name__ == '__main__':
    main()

Packet 13 full of bonuses
Packet 16 full of bonuses
randomized
randomized
randomized
randomized
randomized
randomized
randomized
randomized
randomized
randomized
randomized
randomized
randomized
randomized
randomized
randomized
program done
